In [1]:
# Import necessary modules
import sys
sys.path.append('../fraud_detection')  # Adjust this path based on your working directory
from utils.logging_util import setup_logger

# notebooks/eda_fraud_detection.ipynb

# Import the necessary functions from your project
from model_training import prepare_data, train_logistic_regression , train_random_forest , train_gradient_boosting ,train_decision_tree

from model_evaluation import evaluate_model




In [2]:
data_path = '../data/fraud_datas.csv'
prepared_data = prepare_data(data_path, target_column='fraud_class')

2024-11-15 16:32:57,048 - root - INFO - Loaded data from ../data/fraud_datas.csv
2024-11-15 16:32:57,048 - root - INFO - Loaded data from ../data/fraud_datas.csv
2024-11-15 16:32:58,050 - root - INFO - Encoded column: device_id
2024-11-15 16:32:58,050 - root - INFO - Encoded column: device_id
2024-11-15 16:32:58,091 - root - INFO - Encoded column: source
2024-11-15 16:32:58,091 - root - INFO - Encoded column: source
2024-11-15 16:32:58,127 - root - INFO - Encoded column: browser
2024-11-15 16:32:58,127 - root - INFO - Encoded column: browser
2024-11-15 16:32:58,162 - root - INFO - Encoded column: sex
2024-11-15 16:32:58,162 - root - INFO - Encoded column: sex
2024-11-15 16:32:58,210 - root - INFO - Data successfully split into train and test sets.
2024-11-15 16:32:58,210 - root - INFO - Data successfully split into train and test sets.
2024-11-15 16:32:58,259 - root - INFO - Standardized the data.
2024-11-15 16:32:58,259 - root - INFO - Standardized the data.


In [3]:

# Access the prepared data
X_train = prepared_data['X_train']
X_test = prepared_data['X_test']
y_train = prepared_data['y_train']
y_test = prepared_data['y_test']



In [5]:
# Train and save the Logistic Regression model
logistic_model_path = '../models/logistic_regression_model.pkl'
logistic_results = train_logistic_regression(X_train, X_test, y_train, y_test, logistic_model_path)

# Train and save the Random Forest model
random_forest_model_path = '../models/random_forest_model.pkl'
random_forest_results = train_random_forest(X_train, X_test, y_train, y_test, random_forest_model_path)

# Display results
print(logistic_results)
print(random_forest_results)

2024-11-15 16:41:33,718 - root - INFO - Training model: LogisticRegression.
2024-11-15 16:41:33,718 - root - INFO - Training model: LogisticRegression.
2024-11-15 16:41:33,857 - root - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-15 16:41:33,857 - root - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-15 16:41:33,897 - root - INFO - Model training and evaluation completed successfully.
2024-11-15 16:41:33,897 - root - INFO - Model training and evaluation completed successfully.
2024-11-15 16:41:33,899 - root - INFO - Training model: RandomForestClassifier.
2024-11-15 16:41:33,899 - root - INFO - Training model: RandomForestClassifier.
2024-11-15 16:42:41,692 - root - INFO - Model saved to ../models/random_forest_model.pkl.
2024-11-15 16:42:41,692 - root - INFO - Model saved to ../models/random_forest_model.pkl.
2024-11-15 16:42:43,473 - root - INFO - Model training and evaluation completed successfully.
2024-11-15 16:42:43,473 - root - I

{'model': LogisticRegression(class_weight='balanced', max_iter=2000), 'training_time': 0.13431000709533691, 'roc_auc': np.float64(0.7580767576706717), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      0.64      0.77     27373\n           1       0.17      0.70      0.27      2850\n\n    accuracy                           0.65     30223\n   macro avg       0.56      0.67      0.52     30223\nweighted avg       0.88      0.65      0.72     30223\n'}
{'model': RandomForestClassifier(class_weight='balanced'), 'training_time': 67.59041357040405, 'roc_auc': np.float64(0.7651919710868886), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.98     27373\n           1       1.00      0.54      0.70      2850\n\n    accuracy                           0.96     30223\n   macro avg       0.98      0.77      0.84     30223\nweighted avg       0.96      0.96      0.95     30223\n'}


In [6]:
# # Import necessary functions
# from fraud_detection.model_training import train_logistic_regression, train_decision_tree, train_random_forest, train_gradient_boosting


decision_tree_model_path = '../models/decision_tree_model.pkl'

gradient_boosting_model_path = '../models/gradient_boosting_model.pkl'


# Train and save the Decision Tree model
decision_tree_results = train_decision_tree(X_train, X_test, y_train, y_test, decision_tree_model_path)


# Train and save the Gradient Boosting model
gradient_boosting_results = train_gradient_boosting(X_train, X_test, y_train, y_test, gradient_boosting_model_path)

# Display results for all models

print("\nDecision Tree Model Results:")
print(decision_tree_results)

print("\nGradient Boosting Model Results:")
print(gradient_boosting_results)


2024-11-15 16:42:43,502 - root - INFO - Training model: DecisionTreeClassifier.
2024-11-15 16:42:43,502 - root - INFO - Training model: DecisionTreeClassifier.
2024-11-15 16:42:47,580 - root - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-15 16:42:47,580 - root - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-15 16:42:47,639 - root - INFO - Model training and evaluation completed successfully.
2024-11-15 16:42:47,639 - root - INFO - Model training and evaluation completed successfully.
2024-11-15 16:42:47,641 - root - INFO - Training model: GradientBoostingClassifier.
2024-11-15 16:42:47,641 - root - INFO - Training model: GradientBoostingClassifier.
2024-11-15 16:43:38,363 - root - INFO - Model saved to ../models/gradient_boosting_model.pkl.
2024-11-15 16:43:38,363 - root - INFO - Model saved to ../models/gradient_boosting_model.pkl.
2024-11-15 16:43:38,479 - root - INFO - Model training and evaluation completed successfully.
2024-11-15 16:43:38,47


Decision Tree Model Results:
{'model': DecisionTreeClassifier(), 'training_time': 4.070529937744141, 'roc_auc': np.float64(0.7533836133826328), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      0.94      0.95     27373\n           1       0.50      0.56      0.53      2850\n\n    accuracy                           0.91     30223\n   macro avg       0.73      0.75      0.74     30223\nweighted avg       0.91      0.91      0.91     30223\n'}

Gradient Boosting Model Results:
{'model': GradientBoostingClassifier(), 'training_time': 50.71985197067261, 'roc_auc': np.float64(0.7743369026079611), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.98     27373\n           1       1.00      0.54      0.70      2850\n\n    accuracy                           0.96     30223\n   macro avg       0.98      0.77      0.84     30223\nweighted avg       0.96      0.96      0.95     30223\n'}


In [7]:
# Evaluate all models
logistic_evaluation = evaluate_model(logistic_results, X_test, y_test)
decision_tree_evaluation = evaluate_model(decision_tree_results, X_test, y_test)
random_forest_evaluation = evaluate_model(random_forest_results, X_test, y_test)
gradient_boosting_evaluation = evaluate_model(gradient_boosting_results, X_test, y_test)

# Display results for all models
print("Logistic Regression Model Evaluation:")
print(logistic_evaluation)

print("\nDecision Tree Model Evaluation:")
print(decision_tree_evaluation)

print("\nRandom Forest Model Evaluation:")
print(random_forest_evaluation)

print("\nGradient Boosting Model Evaluation:")
print(gradient_boosting_evaluation)

2024-11-15 16:43:38,507 - root - ERROR - Error during evaluation: 'dict' object has no attribute 'predict'
2024-11-15 16:43:38,507 - root - ERROR - Error during evaluation: 'dict' object has no attribute 'predict'


AttributeError: 'dict' object has no attribute 'predict'